In [1]:
!pip install langchain langchain_openai langsmith pandas langchain_experimental matplotlib langgraph langchain_core duckduckgo-search langchain-community chromadb


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import dotenv
dotenv.load_dotenv()

True

# 커스텀 툴 정의하기

에이전트를 구성할 때, 에이전트가 사용할 수 있는 도구의 목록(Tools)을 제공해야함.

도구는 호출되는 실제 함수 외에도 여러 구성 요소로 이루어져 있음.

| Attribute  | Type  |  Description |
|---|---|---|
| name  | str  | LLM 또는 에이전트에 제공된 도구 세트 내에서 고유(unique)해야 합니다.  |
| description | str  |  도구가 무엇을 하는지 설명합니다. LLM 또는 에이전트에서 문맥으로 사용됩니다. |
| args_schema	| pydantic.BaseModel  | 선택 사항이지만 권장되며, 콜백 핸들러를 사용하는 경우 필수입니다. 예상되는 매개변수에 대한 추가 정보(예: few-shot examples) 또는 검증을 제공하는 데 사용할 수 있습니다. |
| return_direct		| boolean  |  에이전트에만 관련됩니다. True로 설정된 경우, 지정된 도구를 호출한 후 에이전트가 작업을 중지하고 결과를 사용자에게 직접 반환합니다. |


# Creating tools from functions

## @tool decorator
@tool 데코레이터는 사용자 정이 도구(custom tool)를 정의하는 가장 간단한 방법입니다. 이 데코레이터는 기본적으로 함수 이름을 도구 이름으로 사용하지만,
첫 번째 인자로 문자열을 전달하여 이를 재정의할 수 있습니다.
또한, 함수의 docstring 을 도구의 설명으로 사용하므로 반드시 docstring을 제공해야합니다.